In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from citipy import citipy
from random import uniform
from config import api_key

settings = {"units": "metric", "appid": api_key}
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

In [14]:
# Create a df to add city, country, lat, lon, temp, humidty, cloudiness, and wind speed. 
column_names = ["City", "Country", "Latitude", "Longitude", "Temperature", "Humidty", "Cloudiness", "Wind Speed (mph)"]
weather_data = pd.DataFrame(columns=column_names)

## Generate Cities List 

In [15]:
# Radomly generate a list of 1500 cities. Use 10 to test. 
for x in range(10):
    lat, lng = np.random.uniform(low=-90.000, high=90.000), np.random.uniform(low=-180.000, high=180.000)
    city = citipy.nearest_city(lat, lng)
    weather_data = weather_data.append({"City": city.city_name,"Country": city.country_code,}, ignore_index=True)
weather_data

,City,Country,Latitude,Longitude,Temperature,Humidty,Cloudiness,Wind Speed (mph)
0,banda aceh,id,NaN,NaN,NaN,NaN,NaN,NaN
1,punta de piedra,hn,NaN,NaN,NaN,NaN,NaN,NaN
2,albany,au,NaN,NaN,NaN,NaN,NaN,NaN
3,fereydun kenar,ir,NaN,NaN,NaN,NaN,NaN,NaN
4,provideniya,ru,NaN,NaN,NaN,NaN,NaN,NaN
5,hithadhoo,mv,NaN,NaN,NaN,NaN,NaN,NaN
6,carnarvon,au,NaN,NaN,NaN,NaN,NaN,NaN
7,kapaa,us,NaN,NaN,NaN,NaN,NaN,NaN
8,vila,vu,NaN,NaN,NaN,NaN,NaN,NaN
9,bluff,nz,NaN,NaN,NaN,NaN,NaN,NaN


## Perform API Calls

In [16]:
# Loop through the list of cities in the df and get location and weather.
data = []
for index, row in weather_data.iterrows():
    target_city = row["City"]
    query_url = url + "appid=" + api_key + "&units=" + units + "&q="+target_city
    response = requests.get(query_url).json()
    data.append(response)
data

[{'base': 'stations',
  'clouds': {'all': 12},
  'cod': 200,
  'coord': {'lat': 5.56, 'lon': 95.32},
  'dt': 1510769215,
  'id': 1215502,
  'main': {'grnd_level': 996.58,
   'humidity': 98,
   'pressure': 996.58,
   'sea_level': 1022.32,
   'temp': 24.09,
   'temp_max': 24.09,
   'temp_min': 24.09},
  'name': 'Banda Aceh',
  'sys': {'country': 'ID',
   'message': 0.1737,
   'sunrise': 1510702041,
   'sunset': 1510744764},
  'weather': [{'description': 'few clouds',
    'icon': '02n',
    'id': 801,
    'main': 'Clouds'}],
  'wind': {'deg': 74.5029, 'speed': 1.03}},
 {'base': 'stations',
  'clouds': {'all': 76},
  'cod': 200,
  'coord': {'lat': 7.62, 'lon': -71.49},
  'dt': 1510769215,
  'id': 3629604,
  'main': {'grnd_level': 953.7,
   'humidity': 78,
   'pressure': 953.7,
   'sea_level': 1020.82,
   'temp': 26.94,
   'temp_max': 26.94,
   'temp_min': 26.94},
  'name': 'Punta de Piedra',
  'sys': {'country': 'VE',
   'message': 0.1712,
   'sunrise': 1510742253,
   'sunset': 1510784630}

In [17]:
len(data)

10